<a href="https://colab.research.google.com/github/Andriansyah2501/Proyek_Analisis_Sentimen/blob/main/Analisis_Sentiment_skema_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [13]:
from google.colab import drive
drive.mount('/content/drive')

# Memuat dataset CSV dari hasil clustering
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset sentimen/dataset_review_mbg.csv')

# Cek 5 baris pertama buat pastiin data ke-load
print("5 Baris Pertama Dataset:")
print(df.head())

# Cek info dataset (kolom, tipe data, dll.)
print("\nInfo Dataset:")
print(df.info())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
5 Baris Pertama Dataset:
   id                nama_program      lokasi  \
0   1      Pangan Gratis Posyandu    Denpasar   
1   2            Dapur Umum Sehat    Makassar   
2   3  Program Gizi Sehat Sekolah    Makassar   
3   4  Program Gizi Sehat Sekolah  Balikpapan   
4   5      Pangan Gratis Posyandu    Semarang   

                                              review  rating     tanggal  
0  Rasa makanannya enak dan bergizi, anak saya suka.       4  2025-05-04  
1  Program ini luar biasa, makanan lengkap dan be...       5  2025-03-11  
2  Makanan selalu segar dan bergizi, sangat memba...       4  2025-04-14  
3  Tidak ada informasi jelas tentang jadwal distr...       1  2025-05-06  
4  Tidak ada informasi jelas tentang jadwal distr...       2  2024-12-14  

Info Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (

In [14]:
# Label sentimen dari skor rating
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

df['sentiment'] = df['rating'].apply(label_sentiment)
print("Distribusi sentimen:\n", df['sentiment'].value_counts())

# TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['review'].astype(str)).toarray()

# Label encoding
le = LabelEncoder()
y = le.fit_transform(df['sentiment'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluasi
y_pred = model.predict(X_test)
print(f"\n🎯 Akurasi Testing: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# Fungsi prediksi
def prediksi_sentimen(kalimat):
    vector = vectorizer.transform([kalimat]).toarray()
    hasil = model.predict(vector)
    return le.inverse_transform(hasil)[0]

# Contoh prediksi
contoh = "Program ini sangat membantu dan makanannya enak"
print("\n🧠 Prediksi untuk contoh kalimat:")
print(f"Kalimat: {contoh}")
print(f"Sentimen: {prediksi_sentimen(contoh)}")

Distribusi sentimen:
 sentiment
positif    80
negatif    72
netral     48
Name: count, dtype: int64

🎯 Akurasi Testing: 67.50%

📊 Classification Report:
               precision    recall  f1-score   support

     negatif       0.87      0.93      0.90        14
      netral       0.20      0.10      0.13        10
     positif       0.65      0.81      0.72        16

    accuracy                           0.68        40
   macro avg       0.57      0.61      0.58        40
weighted avg       0.61      0.68      0.64        40


🧠 Prediksi untuk contoh kalimat:
Kalimat: Program ini sangat membantu dan makanannya enak
Sentimen: positif
